In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
from plotly.offline import plot

# Load the CSV data
df = pd.read_csv('scopus-July-02-2024.csv')

# Create a graph for co-authorship network
G = nx.Graph()

# Add edges between authors who co-authored papers together
for _, row in df.iterrows():
    authors = row['Authors'].split('; ')
    if len(authors) > 1:
        for i in range(len(authors)):
            for j in range(i + 1, len(authors)):
                G.add_edge(authors[i], authors[j])

# Prepare node positions for Plotly
pos = nx.spring_layout(G, k=0.15, iterations=20)

# Function to abbreviate titles
def abbreviate_title(title, max_length=50):
    return title[:max_length] + "..." if len(title) > max_length else title

# Create nodes trace with detailed hover information
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        color='skyblue',
        symbol='circle',
        line_width=2
    ),
    text=[
        f'<b>{node}</b><br>'
        f'<b>Title:</b> {abbreviate_title(df[df["Authors"].str.contains(node)]["Title"].iloc[0])}<br>'
        f'<b>Year:</b> {df[df["Authors"].str.contains(node)]["Year"].iloc[0]}<br>'
        f'<b>Source title:</b> {df[df["Authors"].str.contains(node)]["Source title"].iloc[0]}<br>'
        f'<b>Cited by:</b> {df[df["Authors"].str.contains(node)]["Cited by"].iloc[0]}<br>'
        f'<b>DOI:</b> {df[df["Authors"].str.contains(node)]["DOI"].iloc[0]}<br>'
        f'<b>Author Keywords:</b> {df[df["Authors"].str.contains(node)]["Author Keywords"].iloc[0]}<br>'
        f'<b>Publisher:</b> {df[df["Authors"].str.contains(node)]["Publisher"].iloc[0]}' 
        for node in G.nodes()
    ],
)

# Create edges trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    mode='lines',
    line=dict(width=0.5, color='#888'),
    hoverinfo='none'
)

# Add edge information to the trace
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# Create figure
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Interactive Co-Authorship Network',
        titlefont_size=16,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)

# Plot and display
plot(fig, filename='co_authorship_network.html')
